# Determinant

In [1]:
minors :: [a] -> [[a]]
minors [ ] = [ ]
minors (x : xs) = xs : map (x :) (minors xs)

In [2]:
minors "abcd"

["bcd","acd","abd","abc"]

In [3]:
det :: [[Integer]] -> Integer
det [[x ]] = x
det xss = foldr1 (-) (zipWith (*) col1 (map det (minors cols)))
          where col1 = map head xss
                cols = map tail xss

In [4]:
matrix = [[2,2,1],[9,2,5],[3,7,3]]
det matrix

-25

In [5]:
matrix = [[2,2],[9,5]]
det matrix

-8

# Convex Hull

In [6]:
import Data.List  

In [7]:
-- Type synonym (basically just an alias)
type Point = [Integer]
type Facet = ([Point], Int)
type Simplex = ([Point], Int)

In [8]:
dimension :: Point -> Int
dimension ps = length ps - 1

orientation :: [Point] -> Int
orientation = fromIntegral . signum . det

In [9]:
facets :: Simplex -> [Facet]
facets (us, b) = zip (minors us) (cycle [b, -b])

minors :: [a] -> [[a]]
minors [] = []
minors (x : xs) = xs : map (x :) (minors xs)

In [10]:
insideCS :: Simplex -> Point -> Bool
insideCS smp p = and [0 <= b * orientation (p : us) | (us, b) <- facets smp]

In [11]:
pts = [[1,2,3], [1,2,2], [-1,1,4]]
orientation pts

1

In [12]:
s = (pts,1)
p = [1,1,2]
insideCS s p

False

## Convex Hull

In [13]:
-- The value of tuples n vs is a list of all n-tuples of vs; that is, all subsequences
-- of vs of length n. The definition of tuples is left as an exercise.
-- tuples :: Int -> [] -> [[]]
tuples n vs = filter (\x -> length x == n) (subsequences vs)

In [14]:
insideCH :: [Point] -> Point -> Bool
insideCH vs p = or [insideCS smp p | smp <- simplexes vs]

simplexes :: [Point] -> [Simplex]
simplexes vs = [(us, b) | us <- tuples (d + 1) vs,
                          let b = orientation us, b /= 0]
               where d = dimension (head vs)

In [15]:
vertices :: Simplex -> [Point]
vertices = sort . fst

In [16]:
insideCH pts p

False

## Find a Simplex

In [17]:
submatrices k vs = map (++[last vs]) (tuples k (init vs))
degenerate k = all (==0) . map det . submatrices k . transpose

In [18]:
findSimplex :: [Point] -> Maybe Simplex
findSimplex [] = Nothing
findSimplex (v : vs) = search (length v - 1) 1 [v] vs

search d k us vs
  | k == d + 1 = Just (us, orientation us)
  | null vs = Nothing
  | degenerate k (v : us) = search d k us (tail vs)
  | otherwise = search d (k + 1) (v : us) (tail vs)
    where v = head vs


In [19]:
findSimplex pts

Just ([[-1,1,4],[1,2,2],[1,2,3]],-1)

## Update

In [20]:
external :: [Simplex] -> [Facet]
external = foldr op [] . sort . concatMap facets

op smp [] = []
op smp (smp' : smps) = if vertices smp == vertices smp' then smps
                       else smp : smp' : smps

In [21]:
visible :: Point -> [Facet] -> [Facet]
visible v fs = [(us, b) | (us, b) <- fs, b * orientation (v : us) < 0]

In [22]:
newSimplex :: Point -> Facet -> Simplex
newSimplex v (us, b) = (v : us, -b)

In [23]:
update :: [Simplex] -> Point -> [Simplex]
update smps v = smps ++ map (newSimplex v) (visible v (external smps))

## Incremental Algorithm

In [24]:
insideCH' :: [Point] -> Point -> Bool
insideCH' vs p = or [insideCS smp p | smp <- partition vs] 

partition :: [Point] -> [Simplex]
partition vs
    = case findSimplex vs of
        Nothing -> []
        Just smp -> foldl update [smp] (vs \\ vertices smp)
        -- Just [smp] -> foldl update [smp] (vs \\ vertices smp)

In [25]:
insideCH' pts p

False

In [26]:
insideCH' pts [1,2,3]

True

## QuickCheck

In [27]:
import Test.QuickCheck

In [28]:
point :: Int -> Gen [Integer]
point d = do {xs <- vector d; return (xs ++ [1])}

points :: Int -> Int -> Gen [[Integer]]
points d 0 = return []
-- This is supposed to be n+1 but it won't compile :-(
points d n = do
 p <- point d
 ps <- points d n
 return (p : ps) 


In [32]:
prop_Hull :: Int -> Int -> Property
prop_Hull d n = forAll (points d (n+1)) $ \vs -> 
                forAll (point d) $ \v -> 
                insideCH vs v /= insideCH' vs v

In [30]:
prop_Hull :: Int -> Int -> Bool
prop_Hull d n = forAll (points d (n+1)) $ \vs -> 
                forAll (point d) $ \v -> 
                insideCH vs v == insideCH' vs v

: 

In [31]:
main = quickCheck(prop_Hull 2 3)

In [ ]:
verboseCheck (prop_Hull 2 3)

Passed:

In [ ]:
quickCheckWith stdArgs { maxSuccess = 10 } (prop_Hull 2 3)

In [ ]:
import Test.QuickCheck

prop_commutativeAdd :: Integer -> Integer -> Bool
prop_commutativeAdd n m = n + m == m + n

In [ ]:
quickCheckWith stdArgs { maxSuccess = 500 } prop_commutativeAdd

In [ ]:
quickCheckWith stdArgs { maxSuccess = 1 } prop_commutativeAdd

In [ ]:
quickCheck prop_commutativeAdd

In [ ]:
 verboseCheck prop_commutativeAdd